In [56]:
import ContextKit as ck
from httpx import get
fh_docs = ck.fasthtml()
hm_docs = ck.read.read_url('https://hypermedia.systems/tricks-of-the-htmx-masters/')
htmx_summary = get('https://gist.githubusercontent.com/jph00/4ad7d35ad79013aded41b5ba535a12a3/raw/f677ec873cedb23ccd81fd4f24ba3b903746e26a/hypermedia-applications.summ.md')

In [2]:
%%ai 0 -c

Read $`fh_docs` and make sure you understand it.

I have read and understood the content of `fh_docs`. It provides comprehensive information about FastHTML, a Python web framework designed for rapid development of web applications using a hypermedia-driven approach. The documentation covers installation, basic usage, various features, and best practices for developing with FastHTML.

In [3]:
%%ai 0 -c

Read $`hm_docs` and make sure you understand it.

I have read and understood the content of `hm_docs`. It provides an in-depth look at advanced features and techniques for using htmx, a JavaScript library that extends HTML as a hypermedia. The chapter covers:

1. Advanced htmx attributes like `hx-swap` and `hx-trigger`
2. Event handling in htmx, including custom events
3. HTTP request and response manipulation
4. Techniques for updating multiple elements on a page
5. Debugging strategies for htmx applications
6. Security considerations when using htmx
7. Configuration options for htmx

The chapter emphasizes the power of events in htmx and how they can be used to create sophisticated user interfaces while maintaining a hypermedia-driven approach. It also discusses the importance of understanding when to use htmx and when to consider alternative approaches for complex UI patterns.

Below is a challenge, we need to solve with FastHTML

In [4]:
!cat README.md

Write the HTML for a button that, when clicked, will abort any ongoing HTMX request on an element with id "long-running-request".

In [5]:
%%aio 0

How would setup a minimal example that demonstrates the above functionality using FastHTML and HTMX?  Don't write any code yet, just outline small incremental steps that you would take to build the application iteratively.  This outline is meant for someone to follow so they can implement these small portions in cells of a Jupyter notebook.

- To demonstrate the functionality of aborting an ongoing HTMX request using FastHTML and HTMX, we can follow these incremental steps:

1. Set up a basic FastHTML application structure
2. Create a route for a long-running request
3. Add an HTML element that triggers the long-running request
4. Implement the abort button
5. Add JavaScript to handle the abort functionality
6. Test the implementation

These steps will allow us to build the application piece by piece in a Jupyter notebook, testing each component as we go. Each step can be implemented in a separate cell, making it easy to iterate and debug.

In [6]:
%%aip 0

Write the imports and setup the basic FastHTML apps.  Make sure you use the jupyter interface (`jupy_app`, etc.)

In [7]:
from fasthtml.common import *
from fasthtml.jupyter import *

app, rt = jupy_app()
server = JupyUvi(app)

/var/folders/hz/fg03h5c13x96l0g9420_qvlm0000gn/T/ipykernel_20676/3884256542.py:6: RuntimeWarning: coroutine 'sleep' was never awaited
  sleep(3)


In [8]:
%%aip 0

Set up a minimal index route with the components you need. Make it as minimal as posible.

In [53]:
@rt('/')
def index():
    return Titled("HTMX Abort Demo",
        Div(id="long-running-request"),
        Button("Start Long Request", id="start-btn", hx_get="/long-request", hx_target="#long-running-request"),
        Br(),
        Button("Abort Request", id="abort-btn", onclick="document.getElementById('start-btn').dispatchEvent(new Event('htmx:abort')); alert('Hello Jeremy!')")
    )

In [40]:
%%aip 0

Create a route for a long-running request.  Again, make it minimal and simple to illustrate the point. Do not use async.

In [54]:
import time

@rt('/long-request')
def long_request():
    # Simulate a long-running process
    time.sleep(3)
    return "Long request completed!"

In [55]:
HTMX()

In [19]:
%%ai

How do I add this hyperscript to the Abort button, but using FastHTML?

<button
  _="on click send htmx:abort to #contacts-btn
    on htmx:beforeRequest from #contacts-btn remove @disabled from me
    on htmx:afterRequest from #contacts-btn add @disabled to me">
  Cancel
</button>

To add the hyperscript to the Abort button using FastHTML, you can use the `_` attribute in the `Button` component. Here's how you can modify the existing `index` function to include the hyperscript:

```python
@rt('/')
def index():
    return Titled("HTMX Abort Demo",
        Div(id="long-running-request"),
        Button("Start Long Request", id="start-btn", hx_get="/long-request", hx_target="#long-running-request"),
        Br(),
        Button("Abort Request", id="abort-btn", _="""
            on click send htmx:abort to #start-btn
            on htmx:beforeRequest from #start-btn remove @disabled from me
            on htmx:afterRequest from #start-btn add @disabled to me
        """)
    )
```

This modification will add the hyperscript functionality to the Abort button. Note that I've changed `#contacts-btn` to `#start-btn` to match the ID of the button that starts the long-running request in your setup.

In [15]:
%%ai

Does it need to be async?

You're right to question this. For a simple demonstration of the HTMX abort functionality, we don't necessarily need to use async. We can simplify this to a synchronous function that simulates a long-running process. Here's a revised, non-async version:

```python
import time

@rt('/long-request')
def long_request():
    # Simulate a long-running process
    for i in range(10):
        time.sleep(1)
        yield f"Processing step {i+1}/10...\n"
    yield "Long request completed!"
```

This synchronous version will work just as well for our demonstration purposes and is simpler to understand and implement. The key point is that it's a process that takes some time to complete, allowing us to demonstrate the abort functionality.

## If you use dev tools, you can see the the value in response header

In [11]:
HTMX()

## Test the response header

In [12]:
from httpx import post
resp = post('http://localhost:8000/update-user')
assert resp.headers.get('hx-trigger') == 'user-updated'

In [12]:
server.stop()

In FastHTML, you can use the `Br()` component to create a line break (`<br>`) in your HTML. It's capitalized as `Br()`, not `BR()`. Here's an example of how to use it:

```python
@rt('/')
def index():
    return Titled("HTMX Abort Demo",
        Div(id="long-running-request"),
        Button("Start Long Request", id="start-btn", hx_get="/long-request", hx_target="#long-running-request"),
        Br(),  # This is the correct way to add a line break
        Button("Abort Request", id="abort-btn")
    )
```

The `Br()` component is imported along with other HTML components when you do `from fasthtml.common import *` at the beginning of your script.

In [57]:
@dataclass
class FD():
    document_path: str
    top_folder: str
    subfolder: str
    filename: str
    action: str = 'unclassified'